### Про анализ социальных сетей и графовую аналитику есть несколько статей на сайте NTA, поэтому перейдем непосредственно к практической части.  При анализе файлов XML были найдены теги, под которыми хранятся данные мобильных контактов. В результате для извлечения необходимых номеров была написана, следующая функция, на языке python:

In [ ]:
import networkx as nx
import os
import re
import sys
import xml.etree.ElementTree as etree


def import_contacts(root):
    # импортирует список контактов из XML (пустые значения отбрасываются)
    contacts = []
    tag = root.find("./MobileDevice ")

    if tag is None:
        return None

    tag = tag.attrib

    if tag.get('Name') == 'PHONEBOOK':
        for phone in root.findall("./MobileDevice /Item/Field/[@Type='FLD_PB_NUMTYPE_MOBILE']"):
            if phone.text is None:
                continue
            else:
                contacts.append(phone.text)
    elif tag.get('Name') == 'AGGREGATED_CONTACTS':
        for phone in root.findall("./MobileDevice /Item/Field/[@Type='FLD_AGGRCONT_NUMTYPE_MOBILE']"):
            if phone.text is None:
                continue
            else:
                contacts.append(phone.text)
    else:
        return None

    return contacts

### Полученные данные необходимо было нормализовать к единому формату телефонных номеров, т.к. где-то они хранятся в написании с 8-ки, где-то с 7, где-то вообще цифры, не похожие на номера телефонов. Для преобразования была написана следующая функция:

In [ ]:
def normalize_phones_list(phones_list):
    """
   приводим № телефонов в единый формат (начинается с 7-ки вместо 8-ки)
   остальные отбрасываются
    """
    i = 0
    phones = []
    while i < len(phones_list):
        phones_list[i] = phones_list[i].replace(u'\xa0', u' ')
        phones_list[i] = re.sub(r'[^0-9]', r'', phones_list[i])
        if len(phones_list[i]) == 11:
            if phones_list[i][0] == '8':
                phones_list[i] = '7' + phones_list[i][1:]
            phones.append(int(phones_list[i]))
        i += 1
    return phones

### Все данные были экспортированы в XML формат с названиями в виде номера телефона владельца, для того чтобы получить эту вершину (источник связей) с его контактами был написан следующий код: 

In [ ]:
def get_owner_phone_from_filename(filename):
    """
    получает № тел. (исп. для указания телефона-владельца) из имени файла TXT\XML (имя файла должно содержать № тел)
   """
    match = re.search(r'(\d{11})\.(xml|txt)', filename)
    if match is not None:
        owner_phone = match[1]
    else:
        owner_phone = None
    return owner_phone

### Для сбора списка контактов со всех файлов и преобразования к необходимому формату была написана следующая функция:

In [ ]:
def xml_to_txt(pathname):
    """
    конвертация всех XML файлов в папке pathname в TXT (каждый № тел. записывается в отдельную строку)
   возвращается список полученных № тел. с группировкой по номерам-владельцам
    """
    filenames = []
    owners_dict = dict()
    path = os.walk(pathname)
    for (dirpath, dirnames, files) in path:
        for filename in files:
            match = re.search(r'(\d{11})\.xml', filename)
            if match is not None:
                filenames.append(match[0])
        break

    try:
        for filename in filenames:
            tree = etree.parse(pathname + '\\' + filename)
            root = tree.getroot()

            if root.tag == 'OFExport':
                contacts_list = import_contacts(root)

            contacts_list = normalize_phones_list(contacts_list)
            with open(pathname + '\\' + get_owner_phone_from_filename(filename) + '.txt', 'w') as f:
                for item in contacts_list:
                    f.write("%s\n" % item)

            owner_phone = int(get_owner_phone_from_filename(filename))
            owners_dict[owner_phone] = contacts_list
    except etree.ParseError as e:
        print("Ошибка при обработке XML-файла:", filename)
        print(e)
        print("Unexpected error:", sys.exc_info()[0])

    return owners_dict

### Далее для формирования графа по вершинам-владельцам и их связям с контактами был написан следующий код:

In [ ]:
def import_from_txt(pathname):
    """
    загрузка № тел. из TXT файлов, расположенных в папке pathname и её подпапок
    имена подпапок используются в качестве установленных сообществ
    номер-владелец берется из имени файла
    (каждый № тел. в файле должен быть в отдельной строке, имя файла из 11 цифр № тел)
    возвращается список полученных № тел. с группировкой по номерам-владельцам
   """
    case = 'without'
    contacts_list = []
    G = nx.Graph()
    for dirName, subdirList, fileList in os.walk(pathname):
        if dirName != pathname:
            match = re.search(r'(TXTdata\\(\w+))', dirName)
            if match is not None:
                case = match[2]

        for filename in fileList:
            try:
                with open(dirName + '\\' + filename) as f:
                    for line in f:
                        contacts_list.append(line.rstrip('\n'))

                contacts_list = normalize_phones_list(contacts_list)
                owner_phone = int(get_owner_phone_from_filename(filename))
                G.add_node(owner_phone, case=case)
                G.add_nodes_from(contacts_list, case=case)
                for phone in contacts_list:
                    G.add_edge(owner_phone, phone)
                contacts_list = []
            except etree.ParseError as e:
                print("Ошибка при обработке TXT-файла:", filename)
                print(e)
            except:
                print("Unexpected error:", sys.exc_info()[0])
    return G

### И наконец запускаем весь скрипт для обработки всех файлов и сохраняем результат в формат graphml:

In [11]:
data = import_from_txt('TXTdata')
nx.write_graphml(data, 'graph.graphml')
sys.exit()

SystemExit: 